![piggy_bank](piggy_bank.jpg)

<br>

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean and store the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to set up a PostgreSQL database to store this campaign's data, designing the schema in a way that would allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split, in order to save separate files based on the tables you will create. It is recommended to use `pandas` for these tasks.

Lastly, you will write the SQL code that the bank can execute to create the tables and populate with the data from the csv files. As the bank are quite strict about their security, you'll save SQL files as multiline string variables that they can then use to create the database on their end. 

You have been asked to design a database that will have three tables:

## client

| column | data type | description | original column in dataset |
|--------|-----------|-------------|----------------------------|
| `id` | `serial` | Client ID - primary key | `client_id` |
| `age` | `integer` | Client's age in years | `age` |
| `job` | `text` | Client's type of job | `job` |
| `marital` | `text` | Client's marital status | `marital` | 
| `education` | `text` | Client's level of education | `education` |
| `credit_default` | `boolean` | Whether the client's credit is in default | `credit_default` |
| `housing` | `boolean` | Whether the client has an existing housing loan (mortgage) | `housing` | 
| `loan` | `boolean` | Whether the client has an existing personal loan | `loan` |

<br>

## campaign

| column | data type | description | original column in dataset |
|--------|-----------|-------------|----------------------------|
| `campaign_id` | `serial` | Campaign ID - primary key | N/A - new column |
| `client_id` | `serial` | Client ID - references `id` in the `client` table | `client_id` |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | `campaign` |
| `contact_duration` | `integer` | Last contact duration in seconds | `duration` |
| `pdays` | `integer` | Number of days since contact in previous campaign (`999` = not previously contacted) | `pdays` |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | `previous` |
| `previous_outcome` | `boolean` | Outcome of the previous campaign | `poutcome` |
| `campaign_outcome` | `boolean` | Outcome of the current campaign | `y` |
| `last_contact_date` | `date` | Last date the client was contacted | A combination of `day`, `month`, and the newly created `year` |

<br>

## economics

| column | data type | description | original column in dataset |
|--------|-----------|-------------|----------------------------|
| `client_id` | `serial` | Client ID - references `id` in the `client` table | `client_id` |
| `emp_var_rate` | `float` | Employment variation rate (quarterly indicator) | `emp_var_rate` |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | `cons_price_idx` |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three month rate (daily indicator) | `euribor3m` |
| `number_employed` | `float` | Number of employees (quarterly indicator)| `nr_employed` |

##### import library

In [376]:
import pandas as pd
import numpy as np
import psycopg2
import csv
from sqlalchemy import create_engine
# Start coding here...

##### Establish the database connection

In [377]:
conn = psycopg2.connect(
    host = "localhost",
    port = 5432,
    user = "postgres",
    password = "postgres",
    database = "subscriber"
)

# Create a cursor object to interact with the database
cursor = conn.cursor()

##### Read in csv

In [378]:
marketing = pd.read_csv("./data/bank_marketing.csv")

In [379]:
marketing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 22 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   client_id       41188 non-null  int64  
 1   age             41188 non-null  int64  
 2   job             41188 non-null  object 
 3   marital         41188 non-null  object 
 4   education       41188 non-null  object 
 5   credit_default  41188 non-null  object 
 6   housing         41188 non-null  object 
 7   loan            41188 non-null  object 
 8   contact         41188 non-null  object 
 9   month           41188 non-null  object 
 10  day             41188 non-null  int64  
 11  duration        41188 non-null  int64  
 12  campaign        41188 non-null  int64  
 13  pdays           41188 non-null  int64  
 14  previous        41188 non-null  int64  
 15  poutcome        41188 non-null  object 
 16  emp_var_rate    41188 non-null  float64
 17  cons_price_idx  41188 non-null 

##### Split into the three tables

In [380]:
client = marketing[["client_id", "age", "job", "marital", "education", 
             "credit_default", "housing", "loan"]]
campaign = marketing[["client_id", "campaign", "month", "day", 
               "duration", "pdays", "previous", "poutcome", "y"]]
economics = marketing[["client_id", "emp_var_rate", "cons_price_idx", 
                "euribor3m", "nr_employed"]]

In [381]:
campaign.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   client_id  41188 non-null  int64 
 1   campaign   41188 non-null  int64 
 2   month      41188 non-null  object
 3   day        41188 non-null  int64 
 4   duration   41188 non-null  int64 
 5   pdays      41188 non-null  int64 
 6   previous   41188 non-null  int64 
 7   poutcome   41188 non-null  object
 8   y          41188 non-null  object
dtypes: int64(6), object(3)
memory usage: 2.8+ MB


##### Rename columns

In [382]:
client.rename(columns={"client_id": "id"}, inplace=True)

campaign.rename(columns={"duration": "contact_duration", 
                         "y": "campaign_outcome", 
                         "campaign": "number_contacts",
                         "previous": "previous_campaign_contacts",
                         "poutcome": "previous_outcome"}, 
                         inplace=True)

economics.rename(columns={"euribor3m": "euribor_three_months", 
                          "nr_employed": "number_employed"}, 
                          inplace=True)

C:\Users\kamin\AppData\Local\Temp\ipykernel_2852\1788826247.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  client.rename(columns={"client_id": "id"}, inplace=True)
C:\Users\kamin\AppData\Local\Temp\ipykernel_2852\1788826247.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  campaign.rename(columns={"duration": "contact_duration",
C:\Users\kamin\AppData\Local\Temp\ipykernel_2852\1788826247.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-vers

##### Clean data

In [383]:
client["education"] = client["education"].str.replace(".", "_")
client["education"] = client["education"].replace("unknown", np.NaN)

client["credit_default"] = client["credit_default"].replace("unknown", np.NaN)

client["housing"] = client["housing"].replace("unknown", np.NaN)

client["loan"] = client["loan"].replace("unknown", np.NaN)

client["job"] = client["job"].str.replace(".", "")

# Change campaign_outcome to binary values
campaign["campaign_outcome"] = campaign["campaign_outcome"].map({"yes": 1, "no": 0})

# Convert poutcome to binary values
campaign["previous_outcome"] = campaign["previous_outcome"].replace("nonexistent", np.NaN)
# Map "success" to 1 and "failure" to 0, and replace NaN with 0
campaign["previous_outcome"] = campaign["previous_outcome"].map({"success": 1, "failure": 0}).fillna(0).astype(int)

# Add campaign_id column
campaign["campaign_id"] = 1

# Capitalize month and day columns
campaign["month"] = campaign["month"].str.capitalize()

# Add year column
campaign["year"] = "2022"

# Convert day to string
campaign["day"] = campaign["day"].astype(str)

# Add last_contact_date column
campaign["last_contact_date"] = campaign["year"] + "-" + campaign["month"] + "-" + campaign["day"]

# Convert to datetime
campaign["last_contact_date"] = pd.to_datetime(campaign["last_contact_date"], format="%Y-%b-%d")

# Drop unneccessary columns
campaign.drop(columns=["month", "day", "year"], inplace=True)

new_order = ['campaign_id', 'client_id', 'number_contacts'] + [col for col in campaign.columns if col not in ['campaign_id', 'client_id', 'number_contacts']]
campaign = campaign[new_order]

C:\Users\kamin\AppData\Local\Temp\ipykernel_2852\1625294631.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  client["education"] = client["education"].str.replace(".", "_")
C:\Users\kamin\AppData\Local\Temp\ipykernel_2852\1625294631.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  client["education"] = client["education"].replace("unknown", np.NaN)
C:\Users\kamin\AppData\Local\Temp\ipykernel_2852\1625294631.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFram

In [384]:
campaign.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   campaign_id                 41188 non-null  int64         
 1   client_id                   41188 non-null  int64         
 2   number_contacts             41188 non-null  int64         
 3   contact_duration            41188 non-null  int64         
 4   pdays                       41188 non-null  int64         
 5   previous_campaign_contacts  41188 non-null  int64         
 6   previous_outcome            41188 non-null  int32         
 7   campaign_outcome            41188 non-null  int64         
 8   last_contact_date           41188 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int32(1), int64(7)
memory usage: 2.7 MB


##### Save tables to individual csv files

In [385]:
client.to_csv("client.csv", index=False)
campaign.to_csv("campaign.csv", index=False)
economics.to_csv("economics.csv", index=False)

##### Store and print database_design

In [386]:
client_table = """CREATE TABLE client
(
    id SERIAL PRIMARY KEY,
    age INTEGER,
    job TEXT,
    marital TEXT,
    education TEXT,
    credit_default BOOLEAN,
    housing BOOLEAN,
    loan BOOLEAN
);

"""

campaign_table = """CREATE TABLE campaign
(
    campaign_id INTEGER,
    client_id SERIAL references client (id),
    number_contacts INTEGER,
    contact_duration INTEGER,
    pdays INTEGER,
    previous_campaign_contacts INTEGER,
    previous_outcome BOOLEAN,
    campaign_outcome BOOLEAN,
    last_contact_date DATE    
);

"""

economics_table = """CREATE TABLE economics
(
    client_id SERIAL references client (id),
    emp_var_rate FLOAT,
    cons_price_idx FLOAT,
    euribor_three_months FLOAT,
    number_employed FLOAT
);

"""

##### SQL scripts for copying data from CSV files

In [387]:
copy_client_sql = f"copy client FROM 'C:/Users/kamin/DataENGPortfolio/Designing a Bank Marketing Database/client.csv' DELIMITER ',' CSV HEADER;"
copy_campaign_sql = f"copy campaign FROM 'C:/Users/kamin/DataENGPortfolio/Designing a Bank Marketing Database/campaign.csv' DELIMITER ',' CSV HEADER;"
copy_economics_sql = f"copy economics FROM 'C:/Users/kamin/DataENGPortfolio/Designing a Bank Marketing Database/economics.csv' DELIMITER ',' CSV HEADER;"


In [388]:
# SQL scripts to drop and create tables
drop_client_table_sql = "DROP TABLE IF EXISTS client CASCADE;"
drop_campaign_table_sql = "DROP TABLE IF EXISTS campaign;"
drop_economics_table_sql = "DROP TABLE IF EXISTS economics;"


##### Execute the SQL scripts to create tables

In [389]:
cursor.execute(drop_client_table_sql)
cursor.execute(drop_campaign_table_sql)
cursor.execute(drop_economics_table_sql)
cursor.execute(client_table)
cursor.execute(campaign_table)
cursor.execute(economics_table)

conn.commit()

##### Execute the SQL scripts to copy data from CSV files

In [390]:
cursor.execute(copy_client_sql)
cursor.execute(copy_campaign_sql)
cursor.execute(copy_economics_sql)

##### Commit again to complete the data insertion

In [391]:
conn.commit()

##### Close the cursor and connection

In [392]:
cursor.close()
conn.close()